# Load libraries

In [4]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import sklearn
import imblearn
import pickle
from joblib import load, dump

from sklearn.metrics import f1_score, classification_report
from imblearn.ensemble import BalancedRandomForestClassifier

In [5]:
# Download features
# !scp -r -P 22334 -i ~/.ssh/id_rsa guanaco.inf.uach.cl:/home/shared/astro/PLAsTiCC/fats_featurs.tar.gz .
# !tar xzvf fats_features.tar.gz

# Get data

In [6]:
# Select seed, load ids and features

seed = 4
with open(f"ids/seed{seed}/maxClass15k/dataset_ids_before_balancing.pkl", "rb") as f:
    lc_ids = pickle.load(f)

features = {}
for subset in ['train', 'validation', 'test']:
    print(subset)
    tmp = []
    
#     tmp2 = lc_ids[subset][:10]
    
#     for lc_id in tmp2:
    for lc_id in lc_ids[subset]:
#         print(lc_id)
#         print("aoisdj")
#         lc_id = lc_id[:2]
        with open("features/fats"+str(int(lc_id))+".pkl", "rb") as f:
            tmp.append(load(f))
    features[subset] = pd.concat(tmp, axis=0)

FileNotFoundError: [Errno 2] No such file or directory: 'ids/seed4/maxClass15k/dataset_ids_before_balancing.pkl'

In [ ]:
print(features["test"].fillna(-1000).values.shape)

print(lc_ids['labels_test'].astype('int').shape)

# Train RF model

In [ ]:
# Train balanced RF
# from sklearn.metrics import f1_score, classification_report
# from imblearn.ensemble import BalancedRandomForestClassifier

rf = BalancedRandomForestClassifier(n_estimators=500, criterion='entropy', replacement=True,
                                    max_depth=10, class_weight='balanced', n_jobs=8) 
                

rf.fit(
    features['train'].fillna(-1000).values, 
    lc_ids['labels_train'].astype('int'),
) 

# save RF trained model
dump(rf, '../../experiments/comparingModels/seed' + str(seed) + '/RF/trainedRF.joblib') 
print("RF model saved")

# preds = rf.predict(features['validation'].fillna(-1000).values)
# print(classification_report(preds, lc_ids['labels_validation'].astype('int')))
# # print(rf.features)          

# Load saved RF model

In [ ]:
# # load file
rf = load('../../experiments/comparingModels/seed' + str(seed) + '/RF/trainedRF.joblib') 

# Get test model predictions

In [ ]:
# assert lc_ids["test"].shape == lc_ids["labels_test"].shape == preds.shape

In [ ]:
# Result on data set
trainPreds = rf.predict(features['train'].fillna(-1000).values)
validPreds = rf.predict(features['validation'].fillna(-1000).values)
testPreds = rf.predict(features['test'].fillna(-1000).values)

# print(trainPreds)
# print(validPreds)
# print(testPreds)

# save results
results = {
    
    # train
    "trainIds": lc_ids["train"],
    "trainLabels": lc_ids["labels_train"],
    "trainPredictions": trainPreds,
    
    # valid
    "validIds": lc_ids["validation"],
    "validLabels": lc_ids["labels_validation"],
    "validPredictions": validPreds,
    
    # test
    "testIds": lc_ids["test"],
    "testLabels": lc_ids["labels_test"],
    "testPredictions": testPreds,
}

# save object
a_file = open("../../experiments/comparingModels/seed" + str(seed) + "/RF/predictionsRF.pkl", "wb")
pickle.dump(results, a_file)
a_file.close()

# # print(preds)
# print(classification_report(preds, lc_ids['labels_test'][:10].astype('int')))

In [ ]:
# load files
# a_file = open("../../experiments/comparingModels/" + "/testPredictionsRF.pkl", "rb")
# output = pickle.load(a_file)
# print(output["testPredictions"])

In [ ]:
# # Features sorted by importance
# features['train'].columns.values[np.argsort(rf.feature_importances_)[::-1]]